In [1]:
import os

In [2]:
%pwd

'e:\\ML_Projects\\Chicken-Disease-Classification-\\research'

In [3]:
#To change the dir

os.chdir("../")

In [4]:
%pwd

'e:\\ML_Projects\\Chicken-Disease-Classification-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from chicken_classifier.constants import *
from chicken_classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        

In [8]:
import os
import urllib.request as request
import zipfile
from chicken_classifier import logger
from chicken_classifier.utils.common import get_size


In [9]:
class DataIngstion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    
    #downloading the data    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            
            logger.info(f"{filename} downloaded with info: \n{headers}")
            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    #unzipping the file
    def extract_zip_file(self):
        """
        Unzips the file which is availabele
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
#Data Ingestion pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngstion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2023-07-24 18:07:23,413: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-24 18:07:23,418: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-24 18:07:23,418: INFO: common: created directory at: artifacts]
[2023-07-24 18:07:23,418: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-24 18:08:00,013: INFO: 3760326901: artifacts/data_ingestion/data.zip downloaded with info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8BBC:5A80B:13CCD:17CED:64BE7084
Accept-Ranges: bytes
Date: Mon, 24 Jul 2023 12:37:26 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830024-CCU
X-Cache: MISS
X-Cach